In [127]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [128]:
data=pd.read_csv('train_call.tsv', sep='\t')

In [129]:
data = data.transpose()

In [130]:
data2 = data
data = data.drop(data.index[[1, 2, 3]])

In [131]:
samples = pd.read_csv('train_clinical.txt', sep='\t')
samples = samples.set_index('Sample', drop=False).rename_axis(None)

In [132]:
data_simplified = data.drop("Chromosome")
data_simplified.head()

,0,1,2,3,4,5,6,7,8,9,...,2824,2825,2826,2827,2828,2829,2830,2831,2832,2833
Array.129,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,0,1,1,1,1,1
Array.34,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.67,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.24,0,0,0,0,0,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
Array.22,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [133]:
data_simplified.head()

,0,1,2,3,4,5,6,7,8,9,...,2824,2825,2826,2827,2828,2829,2830,2831,2832,2833
Array.129,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,0,1,1,1,1,1
Array.34,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.67,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.24,0,0,0,0,0,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
Array.22,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [134]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [135]:
features = list(data.columns)

newsamples = samples["Subgroup"]

In [145]:
X = data_simplified[features]
y = newsamples

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [146]:
rf = RandomForestClassifier(random_state=0)

rf.fit(X_train,y_train)

y_pred = rf.predict(X_test)

print("Classification report:\n", classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

Classification report:
               precision    recall  f1-score   support

       HER2+       1.00      0.86      0.92         7
         HR+       0.60      0.86      0.71         7
  Triple Neg       0.75      0.50      0.60         6

    accuracy                           0.75        20
   macro avg       0.78      0.74      0.74        20
weighted avg       0.78      0.75      0.75        20

Confusion matrix:
 [[6 1 0]
 [0 6 1]
 [0 3 3]]


In [147]:
# Compute the feature importances
importances = rf.feature_importances_

important_features_dict = {}
for idx, val in enumerate(rf.feature_importances_):
    important_features_dict[idx] = val

important_features_list = sorted(important_features_dict,
                                 key=important_features_dict.get,
                                 reverse=True)

print(important_features_list[:60])

[2184, 2213, 2219, 2206, 851, 853, 1904, 1068, 2211, 301, 764, 1059, 2108, 2026, 833, 692, 385, 2024, 68, 2017, 2116, 856, 1656, 471, 854, 998, 623, 1883, 843, 757, 2162, 837, 743, 2818, 2751, 1678, 1663, 2205, 1910, 2419, 1972, 855, 2756, 478, 860, 1644, 766, 2203, 174, 2208, 861, 721, 389, 2207, 835, 790, 1732, 2198, 476, 1667]


In [139]:
new_features_df = data_simplified[[2184, 2213, 2219, 2206, 851, 853, 1904, 1068, 2211, 301, 764, 1059, 2108, 2026, 833, 692, 385, 2024, 68, 2017, 2116, 856, 1656, 471, 854, 998, 623, 1883, 843, 757, 2162, 837, 743, 2818, 2751, 1678, 1663, 2205, 1910, 2419, 1972, 855, 2756, 478, 860, 1644, 766, 2203, 174, 2208, 861, 721, 389, 2207, 835, 790, 1732, 2198, 476, 1667]
]

In [140]:
#NEW RANDOM FOREST

In [152]:
X = new_features_df
y = newsamples

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [153]:
rf = RandomForestClassifier(random_state=0)

rf.fit(X_train,y_train)

y_pred = rf.predict(X_test)

print("Classification report:\n", classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

Classification report:
               precision    recall  f1-score   support

       HER2+       1.00      1.00      1.00         4
         HR+       0.88      1.00      0.93         7
  Triple Neg       1.00      0.75      0.86         4

    accuracy                           0.93        15
   macro avg       0.96      0.92      0.93        15
weighted avg       0.94      0.93      0.93        15

Confusion matrix:
 [[4 0 0]
 [0 7 0]
 [0 1 3]]


In [154]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
 # define the evaluation procedure
 cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
 # evaluate the model and collect the results
 scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
 return scores

In [155]:
evaluate_model(rf, X, y)

array([0.9, 0.9, 1. , 0.8, 0.8, 0.9, 0.5, 0.8, 0.9, 1. , 0.9, 0.9, 0.9,
       0.8, 0.7, 0.7, 1. , 0.8, 0.5, 0.9, 0.9, 0.8, 1. , 0.9, 0.8, 0.9,
       0.8, 0.9, 0.8, 0.7])